In [2]:
import sys
sys.path.append('C:/Users/Billy/Documents/PRISMO/Backtester')
from multiprocessing import Process
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
import random
from mathCode.johansenMain import coint_johansen
from tqdm import tqdm_notebook
from IPython.display import clear_output
import itertools
import datetime
import os
from base_model_backtest import backtest
import random
from pandas_datareader import data as pdr
import logging
from pandas.plotting import register_matplotlib_converters
from functionSource import *

register_matplotlib_converters()
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
plt.style.use(['ggplot'])


from models.Cointegrated_Pairs_1 import cointegrated_pairs_1


        
global toTrade
global price
global df
global results
global runDate


In [128]:
df = pd.read_csv('D:/Machine_learning_finance/raw_data/ewaewcige.csv')
base = datetime.datetime.today()
date_list = [base + datetime.timedelta(days=x) for x in range(len(df))]
df.index = date_list
prices = np.array([df.ewc,df.ewa]).T

features = df.columns
features = [ 'ewc', 'ewa', 'ige']
toTrade = features

In [129]:
pct_shorted = pickle.load( open('D:/PRISMO/historicalData/shorted/df_percent.pickle','rb'))

In [130]:
toTrade = ['TLS.AX']
data = pickle.load(open('D:/PRISMO/historicalData/data/asx200nobiasFINAL.pickle','rb'))
data_close = data.loc[(slice(None), toTrade)].xs('CLOSE',axis=1,level=1).dropna()
data_open = data.loc[(slice(None), toTrade)].xs('OPEN',axis=1,level=1).dropna()

In [131]:
from pandas_datareader import data

# Only get the adjusted close.
snp200 = data.DataReader("^AXJO",data_source='yahoo')



In [132]:
snp200

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-03,4888.700195,4862.299805,4864.899902,4876.299805,0.0,4876.299805
2010-01-04,4930.700195,4887.299805,4887.299805,4924.299805,0.0,4924.299805
2010-01-05,4938.500000,4915.299805,4927.700195,4921.399902,0.0,4921.399902
2010-01-06,4938.100098,4898.899902,4922.799805,4899.399902,0.0,4899.399902
2010-01-07,4937.899902,4900.299805,4900.299805,4912.100098,0.0,4912.100098
2010-01-10,4955.100098,4921.700195,4921.899902,4950.700195,0.0,4950.700195
2010-01-11,4940.000000,4893.100098,4938.600098,4899.500000,0.0,4899.500000
2010-01-12,4887.000000,4852.899902,4886.299805,4868.100098,0.0,4868.100098
2010-01-13,4920.500000,4875.100098,4876.000000,4898.000000,0.0,4898.000000


In [133]:
# At time t we want: price_close(t) = m_lags:       price_open(t) +price_close(t-1) + price_open(t-1) + price(close(t-2)) ... 
#                                                    + precentage_shorted(t-1)
#                                    n_lags:         + snp_open(t) + snp_close(t-1) + snp_open(t-1)+...

m_=2
n_=2

In [134]:
df = data_close.copy()
df.columns = ['asset_close']
df['asset_open'] = data_open
df['index_close'] = snp200['Close']
df['index_open'] = snp200['Open']

In [135]:
#Lags for the close datafram

for m in range(1,m_+1):
    df['asset_close_lag_'+str(m)] = df['asset_close'].shift(m)
    df['asset_open_lag_'+str(m)] = df['asset_open'].shift(m)

for n in range(1,n_+1):
    df['index_close_lag_'+str(n)] = df['index_close'].shift(n)
    df['index_open_lag_'+str(n)] = df['index_open'].shift(n)

In [136]:
df['percent_shorted'] = pct_shorted[toTrade[0][:-3]]

In [168]:
features_ = [x for x in df.columns if x != 'index_close']
df = df[features_].dropna().astype(float)

In [191]:
ve_values = []
delta_values = []
for ve_ in range(1,10):
    order = round(0.1**(ve_+1), 12)
    for i in range(1,10):
        ve_values.append(round(i*order*100,12))
        delta_values.append(round(i*order,12))

# ve_values = [8e-05]
# delta_values = [8e-11]
results = []
for delta in delta_values:
    for ve in ve_values:
        close = df.copy()
        
        toTrade = features_
        features = features_
        yhat = [np.matrix(0)]           #stores the measurement prediction of the linear model y (t) = x(t)*β(t) + e(t). Store for each time step.

        #The variance in the measurement prediction. Store for each time step.
        Q = []

        #The prediction error at each time step.
        e = []
        #Store the beta values
        beta_list = [np.matrix([[float(0)]]*(len(features)))]   # asset_close(t) + open(t) + index_open + lags(m)*2 * langs(n)*2 + const
        #Covariance Arrays,  measuring the covariance of the error of the hidden variable, β,  estimates
        #For clarity, we denote R(t|t) by P, and R(t|t-1) as R. R(t | t − 1) is cov(β(t) − βhat(t|t-1))
        R = np.zeros([len(features),len(features)])
        P = np.zeros([len(features),len(features)])

        #delta is a pre-defined value, but can be trained from the dataset, delta of 0 means we're just doing linear regression
        Vw = delta/(1-delta)*np.diag(np.ones(len(features)))


        #Decimal Places should be assigned globally for transparency

        #Store the hedge ratios
        hedges = []

        # # For validation
        # # DELETE FOR SPEED
        # self.yhat_val = []
        y = []
        yhat_list = []
        yhatPlusQ = []
        yhatMinusQ = []
        # self.Q = []

        hedges = []

        #The underlying time series, e.g. we use the snp500 for the ASX if we're testing aussie assets.

        pred_close = []
        actual = []
        for row in df.iterrows():
            row = list(list(row)[1])
            y = np.matrix(row[0])
            x = []
            for x_element in row[1:]:
                x.append([x_element])
            x.append([(float(1))])
            x = np.matrix(x)
            
            


            beta = beta_list[-1]
            R = P + Vw
            
            #Estimate
            yhat = np.dot(x.T, beta)
            e_t = y - yhat 
            Q_t = np.dot( np.dot(x.T, R) , x ) + ve
            
            
            
            pred_close.append(yhat.tolist()[0][0])
            actual.append(y.tolist()[0][0])
            #Update
            K = np.matrix(np.dot(R, x)/Q_t)
            beta = beta + K*e_t
            P = R - np.dot(np.dot(K, x.T), R)
            yhat_list.append(yhat.tolist()[0][0])
            e.append(e_t.tolist()[0][0])
            Q.append(Q_t.tolist()[0][0])
            beta_list.append(beta)

        #     beta = beta[-1]
        

        close.loc[slice(None),'yhat'] =  yhat_list
        close.loc[slice(None),'Q'] =  Q

        # if np.mean(close['e'])>np.sqrt(np.mean(close['Q'])):
        # #                 print('means',np.mean(close['e']), np.sqrt(np.mean(close['Q'])))
        # #             if the average error is greater than the average deviation the parameters are fucked. No point going further
        #     break

        close.loc[slice(None),'LongEntry'] = close['yhat']-np.sqrt(close['Q'])*0.2>close['asset_open']
        close.loc[slice(None),'ShortEntry'] = close['yhat']+np.sqrt(close['Q'])*0.2<close['asset_open']

        numUnitsLong = np.array([[0]]*len(close))
        numUnitsShort = np.array([[0]]*len(close))

        numUnitsLong[np.array(close['LongEntry'])] = 1

        numUnitsShort[np.array(close['ShortEntry'])] = -1

        close.loc[slice(None),'short'] = numUnitsShort
        close.loc[slice(None),'long'] =  numUnitsLong

        close.loc[slice(None),'numUnits'] = close['long'] + close['short']

        close.loc[slice(None),'numUnits'+toTrade[0]] = close['numUnits']

        close.loc[slice(None),'positions'+toTrade[0]] = close['numUnits'+toTrade[0]]*close[toTrade[0]]


        close.loc[slice(None),'pnl'] =sum([ close[asset].pct_change()*close['positions'+asset].shift(1) for asset in [toTrade[0]]])


        close.loc[slice(None),'returns'] = (close['pnl'])/(np.abs(close['positions'+toTrade[0]]) +np.abs(close['positions'+toTrade[0]])).shift(1)
        close.loc[slice(None),'returns']= close['returns'][close['returns']<0.1]
        
        try:
            numTrades = len(close[close['numUnits']!=0])
            if numTrades > 50:

                results.append([delta, ve,np.cumsum(close['returns'].fillna(0)).iloc[-1]])

        except Exception as e:
            print(e)


In [190]:
results

[]

In [88]:
[np.matrix([[0]]*(m_*2+n_+1))]

[matrix([[0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0]])]